In [4]:
import pandas as pd
from bs4 import BeautifulSoup

In [21]:
df_q = pd.read_excel("질문내용.xlsx")
df_a = pd.read_excel("등록된 답변.xlsx")
df_ad = pd.read_excel("등록된 답변에 달린 댓글.xlsx")

In [22]:
# 질문내용부터 분석한다.
# 일단 내용이 어떻게 생겼는지 파악해보자.
df_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 83 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   분야           4746 non-null   object
 1   질문명          4746 non-null   object
 2   등록일          4746 non-null   object
 3   등록자          4746 non-null   object
 4   첨부           3928 non-null   object
 5   답변수          4746 non-null   int64 
 6   조회수          4746 non-null   int64 
 7   내용           4746 non-null   object
 8   Unnamed: 8   4746 non-null   object
 9   Unnamed: 9   424 non-null    object
 10  Unnamed: 10  266 non-null    object
 11  Unnamed: 11  168 non-null    object
 12  Unnamed: 12  95 non-null     object
 13  Unnamed: 13  58 non-null     object
 14  Unnamed: 14  44 non-null     object
 15  Unnamed: 15  31 non-null     object
 16  Unnamed: 16  26 non-null     object
 17  Unnamed: 17  20 non-null     object
 18  Unnamed: 18  18 non-null     object
 19  Unnamed: 19  16 non-null   

In [ ]:
df_q[df_q["Unnamed: 9"].notnull()]

In [18]:
df_q.loc[df_q["Unnamed: 33"].notnull(), ["Unnamed: 33", "Unnamed: 34", "Unnamed: 35", "Unnamed: 36"]]

,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36
2226,"""ar"": 0, ""sc"": 1, ""sr"": 1, ""sw"": 3736, ""sh"": ...",", ""di"": false, ""bf"": ""01D7D05F8C11B44000000528...",""": false, ""pr"": false, ""ed"": false, ""di"": fals...","""ln"": """", ""tc"": { ""he"": false, ""hm"": false, ""..."
3115,"109D92F11000000289"", ""ac"": 3, ""ar"": 7, ""sc"": 1...","295"", ""si"": ""01D659109D93182000000296"", ""bf"": ...","sc"": 1134, ""ts"": 8000, ""ms"": """", ""os"": """", ""gl...","} ], [ { ""so"": ""01D659109D92F1100000027B"", ""l..."
3146,"00000F9"", ""ac"": 3, ""ar"": 4, ""sc"": 1, ""sr"": 1, ...",""", ""ln"": """", ""tc"": { ""he"": false, ""hm"": false,...","""2,558,850"" } ] } ] } , ""01D63EC0FE817BF00000...","r"": false, ""ed"": false, ""di"": false, ""bf"": ""01..."


구조를 보니 분야, 질문명, 등록일, 등록자, 첨부, 답변수, 조회수, 내용 까지는 특별한 문제는 없고, "Unnamed: 8"부터 "Unnamed: 82" 까지의 컬럼이 질문내용이라고 판단된다.

컬럼이 많은 이유는 아마도 글자수가 일정 수 이상이 되면 다음 컬럼으로 넘겨서 기록한 듯 하다. 따라서 "Unnamed: 8"부터 "Unnamed: 82" 까지의 컬럼을 하나로 합쳐서 질문내용_html 컬럼에 저장하고, 그 안에서 내용을 추출해보기로 하자.

In [23]:
# "Unnamed: 8"부터 "Unnamed: 82"까지의 컬럼을 하나로 합쳐서 질문내용_html에 저장
df_q['질문내용_html'] = df_q.loc[:, 'Unnamed: 8':'Unnamed: 82'].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

In [26]:
# "Unnamed" 로 시작하는 컬럼들을 없앤다
selected_columns = ['분야', '질문명', '등록일', '등록자', '첨부', '답변수', '조회수', '질문내용_html']
df_q1 = df_q[selected_columns]

In [27]:
df_q1.head()

,분야,질문명,등록일,등록자,첨부,답변수,조회수,질문내용_html
0,신고사건(감독),"영업양도 및 근로관계 인수의 경우, 민형사책임 관련",2024-08-20 11:09:16.0,김태준,,1,66,"<html xml:lang=""ko"" lang=""ko"">\r\n<head>\r\n<s..."
1,신고사건(감독),사용자의 직장 내 괴롭힘에서의 사용자,2024-08-20 09:51:45.0,이도영,,0,98,"<html xml:lang=""ko"" lang=""ko"">\r\n<head>\r\n<s..."
2,신고사건(감독),기간제이자 단시간 근로자 관련 관공서 휴일 질의,2024-08-19 17:10:00.0,장용우,,1,167,"<html xml:lang=""ko"" lang=""ko"">\r\n<head>\r\n<s..."
3,사업장감독(감독),시정지시서가 반송되었을 경우,2024-08-19 16:27:03.0,정현섭,,1,266,"<html xml:lang=""ko"" lang=""ko"">\r\n<head>\r\n<s..."
4,신고사건(감독),임금채권 소멸 시효 산정 문의,2024-08-19 14:07:40.0,백현우,,1,188,"<html xml:lang=""ko"" lang=""ko"">\r\n<head>\r\n<s..."


In [33]:
df_q1['분야'].value_counts()

분야
신고사건(감독)     4001
사업장감독(감독)     542
감독            170
인·허가(감독)       33
Name: count, dtype: int64

In [46]:
df_q1.iloc[3867:3869]

,분야,질문명,등록일,등록자,첨부,답변수,조회수,질문내용_html
3867,신고사건(감독),시급으로 급여 지급시,2018-03-27 20:28:07.0,김점자,,2,262,"<html xml:lang=""ko"" lang=""ko"">\r\n<head>\r\n<t..."
3868,신고사건(감독),사용증명서 규정 관련입니다,2018-03-20 14:10:10.0,노소영,NaN,1,217,<p>근무중인 근로자가 산재보상문제를 상의하기 위해 </p>\r\n<p>호봉승급자료...


In [48]:
df_q1.iloc[3868].질문내용_html

'<p>근무중인 근로자가 산재보상문제를 상의하기 위해 </p>\r\n<p>호봉승급자료, 퇴직금산정자료, 취업규칙서를 요청합니다.</p>\r\n<p>취업규칙서는 사용증명서 포함되는 자료가 아니고,</p>\r\n<p>위 두개의 자료는 근기법39조에 의해 사용자는 즉시 교부해야 할까요?</p>\r\n<p>아니면 재취업용이 아니므로 발급의무가 없는지요?</p>\r\n<p>위 모든 서류가 사용자 발급의무가 없다면 그 근거는 어떤것으로 민원인에게 회시하면 \r\n좋을까요?</p>\r\n<p>&nbsp;</p>\r\n<p>&nbsp;</p>\r\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [8]:
df1['질문내용_html']

0       <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...
1       <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...
2       <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...
3       <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...
4       <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...
                              ...                        
4741    <p>또 생각나는게 있어서 올립니다.</p>\r\n<p>어떻게 생각하면 간단한 것 ...
4742    <p>좋은 공간이 생겼네요. </p>\r\n<p>평소에 궁금하던 것들에 대해&nbs...
4743    <p>회사에서&nbsp;근무하는 근로자가 근로자인지 아닌지 판단할 수 있는 기준을 ...
4744    <p>비정규직에 대한 차별처우 금지영역, 합리적 차별 내용, 차별시정절차 등 차별시...
4745                <p>일용직 주휴수당지급관련 답변을 구합니다.</p>\r\n    
Name: 질문내용_html, Length: 4746, dtype: object

'질문내용_html' 컬럼을 보니 어떤 컬럼은 <html> 태그부터 시작하고, 어떤 컬럼은 <body> 컬럼 안쪽의 내용만 기록되어 있음. 데이터프레임 기준으로 3868번 컬럼, 날짜 기준으로는 2018년 3월 20일 이전 데이터는 <body> 태그 안의 내용만 기록되어 있음.
그래서 <body> 안의 내용만 정리하는 함수를 만들어서 '질문내용_body' 컬럼에 저장해본다.

In [49]:
# <body> 태그 내용을 그대로 추출하는 함수
def extract_body_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    if soup.body:  # <body> 태그가 있으면 해당 내용을 추출
        return str(soup.body)
    else:  # <body> 태그가 없으면 전체 내용을 반환
        return html

# 새로운 컬럼에 저장
df_q1['질문내용_body_html'] = df_q1['질문내용_html'].apply(extract_body_html)

# 결과 확인
print(df_q1[['질문내용_html', '질문내용_body_html']].head())

                                           질문내용_html  \
0  <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...   
1  <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...   
2  <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...   
3  <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...   
4  <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...   

                                      질문내용_body_html  
0  <body>\n<p>안녕하세요, 신고사건 진행중 궁금한 점이 생겨 의견을 여쭈고자 ...  
1  <body>\n<p>학교법인 산하 대학교의 총장을 상대로</p>\n<p>직장 내 괴...  
2  <body>\n<p>관공서의 휴일 적용 관련하여 질의가 들어와 다른 감독관님들의 의...  
3  <body>\n<p>시정지시서가 사업장의 폐문부재 등으로 반송되었을 경우, 범죄인지...  
4  <body>\n<p>질문드립니다.</p>\n<p><br/></p>\n<p><span...  


C:\Users\barkl\AppData\Local\Temp\ipykernel_8232\2351271145.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q1['질문내용_body_html'] = df_q1['질문내용_html'].apply(extract_body_html)


In [50]:
def ensure_body_tags(html):
    soup = BeautifulSoup(html, 'html.parser')
    # <body> 태그가 있으면 그대로 반환
    if soup.body:
        return str(soup.body)
    else:
        # <body> 태그를 감싸서 반환
        return f"<body>{html}</body>"

# 새로운 컬럼에 저장
df_q1['질문내용_body_html'] = df_q1['질문내용_html'].apply(ensure_body_tags)

# 결과 확인
print(df_q1[['질문내용_html', '질문내용_body_html']].head())

                                           질문내용_html  \
0  <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...   
1  <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...   
2  <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...   
3  <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...   
4  <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...   

                                      질문내용_body_html  
0  <body>\n<p>안녕하세요, 신고사건 진행중 궁금한 점이 생겨 의견을 여쭈고자 ...  
1  <body>\n<p>학교법인 산하 대학교의 총장을 상대로</p>\n<p>직장 내 괴...  
2  <body>\n<p>관공서의 휴일 적용 관련하여 질의가 들어와 다른 감독관님들의 의...  
3  <body>\n<p>시정지시서가 사업장의 폐문부재 등으로 반송되었을 경우, 범죄인지...  
4  <body>\n<p>질문드립니다.</p>\n<p><br/></p>\n<p><span...  


C:\Users\barkl\AppData\Local\Temp\ipykernel_8232\817732237.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q1['질문내용_body_html'] = df_q1['질문내용_html'].apply(ensure_body_tags)


In [53]:
df_q1.iloc[3869].질문내용_body_html

'<body><p>(사업장 운영 실태)</p>\r\n<p>A 디자인 회사는 서울에 본점을 두고 지방 백화점 등에 매니저와 수수료를 지급하는 \r\n계약을 하고</p>\r\n<p>매장을 운영하고 있음. </p>\r\n<p>그런데 매장운영이 인력이 필요하여 매니저와 거의 같은 일을 하는 직원 1명을 \r\n두고 아르바이트 &nbsp;직원들도 사용을 하고 있다.</p>\r\n<p>(문제제기)</p>\r\n<p>매니저가 데리고 와서 같이 일하던 직원이 2년을 근무하다 퇴사를 하면서 퇴직금을 \r\n&nbsp;A 디자인 회사</p>\r\n<p>본사 대표이사를 상대로 문제를 제기함</p>\r\n<p>(조사내용)</p>\r\n<p>0.&nbsp;매니저와 직원 주장</p>\r\n<p>&nbsp;&nbsp;- 백화점 매장이라는 것이 본사의 관리하기 매니저는 수수료만을 \r\n받는다. </p>\r\n<p>&nbsp;- 퇴직금을 요구하는 직원의 임금도 &nbsp;통장은 매니저 통장으로 지급을 \r\n하였지만 &nbsp;사실상 본사에서</p>\r\n<p>&nbsp;&nbsp;&nbsp;측정하였고 &nbsp;매니저 통장으로 월임금을 입금을 시켜주면 \r\n곧바로 &nbsp;직원 계좌로 이체를 함</p>\r\n<p>&nbsp;- &nbsp;퇴직금을 요구하는 직원이 처음 근무한 2개월은 본사에서 직접 \r\n직원의 계좌로 월임금을 입금</p>\r\n<p>&nbsp;&nbsp;&nbsp;&nbsp;해주다 갑자기 3개월째부터 매니저 통장으로 넣어주어서 \r\n매니저가 바로 넣어주지 왜 본인계좌로</p>\r\n<p>&nbsp;&nbsp;&nbsp;&nbsp;입금을 시키냐고 하자 관리자가 퇴직금 문제 때문이라고 \r\n이야기를 했다고 주장도 함</p>\r\n<p>0 본 사업장 주장</p>\r\n<p>&nbsp;- 매니저는 본 사업장의 직원이 아니다</p>\r\n<p>&nbsp;- 매니저가 매장을 운영하는데 있어서 직원 1명 정도의 인건비는 지원을 \r\n해 주는 차원이다 </p>\r\n<p>&

In [54]:
def extract_table_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')  # <table> 태그를 찾음
    return str(table) if table else None  # <table> 내용이 있으면 반환, 없으면 None

# 새로운 컬럼에 <table> 태그 내용 저장
df_q1['질문내용_table'] = df_q1['질문내용_body_html'].apply(extract_table_content)

# 결과 확인
print(df_q1[['질문내용_body_html', '질문내용_table']].head())

                                      질문내용_body_html 질문내용_table
0  <body>\n<p>안녕하세요, 신고사건 진행중 궁금한 점이 생겨 의견을 여쭈고자 ...       None
1  <body>\n<p>학교법인 산하 대학교의 총장을 상대로</p>\n<p>직장 내 괴...       None
2  <body>\n<p>관공서의 휴일 적용 관련하여 질의가 들어와 다른 감독관님들의 의...       None
3  <body>\n<p>시정지시서가 사업장의 폐문부재 등으로 반송되었을 경우, 범죄인지...       None
4  <body>\n<p>질문드립니다.</p>\n<p><br/></p>\n<p><span...       None


C:\Users\barkl\AppData\Local\Temp\ipykernel_8232\4123675941.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q1['질문내용_table'] = df_q1['질문내용_body_html'].apply(extract_table_content)


In [56]:
df_q1['질문내용_table'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4746 entries, 0 to 4745
Series name: 질문내용_table
Non-Null Count  Dtype 
--------------  ----- 
69 non-null     object
dtypes: object(1)
memory usage: 37.2+ KB


In [19]:
# <body> 태그 내용만 추출하는 함수
def extract_body(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    body = soup.body  # <body> 태그를 찾음
    if body:        
        return body
    else:
        return html_content

# 데이터프레임에 새로운 컬럼 추가
df1['질문내용_body'] = df1['질문내용_html'].apply(extract_body)

In [57]:
def extract_body_as_text_with_table(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    # 테이블 부분을 먼저 처리
    def extract_table_text(table):
        rows = []
        for tr in table.find_all('tr'):
            cells = [td.get_text(strip=True) for td in tr.find_all(['td', 'th'])]
            rows.append("\t".join(cells))  # 열을 탭으로 구분
        return "\n".join(rows)  # 행을 줄바꿈으로 구분

    final_text = []

    # <body> 태그에서 모든 자식 요소를 순차적으로 처리
    for element in soup.body.children:
        if element.name == 'table':
            # <table>일 경우, 테이블 텍스트로 변환
            final_text.append(extract_table_text(element))
        elif element.name in ['p', 'br', 'ul', 'ol', 'li']:
            # 텍스트가 포함된 태그(p, br, ul, ol, li 등)에 대해서는 그대로 텍스트 추가
            final_text.append(element.get_text(strip=True))
    
    # 결과로 하나의 문자열을 반환
    return "\n".join(final_text)

# 새로운 컬럼 '질문내용_final'에 저장
df_q1['질문내용_final'] = df_q1['질문내용_body_html'].apply(extract_body_as_text_with_table)

C:\Users\barkl\AppData\Local\Temp\ipykernel_8232\4051041936.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q1['질문내용_final'] = df_q1['질문내용_body_html'].apply(extract_body_as_text_with_table)


In [63]:
df_q1.to_csv("output.csv", encoding='utf-8-sig')

PermissionError: [Errno 13] Permission denied: 'output.csv'

In [20]:
df1['질문내용_body']

0       [\n, [안녕하세요, 신고사건 진행중 궁금한 점이 생겨 의견을 여쭈고자 글을 작성...
1       [\n, [학교법인 산하 대학교의 총장을 상대로], \n, [직장 내 괴롭힘을 당했...
2       [\n, [관공서의 휴일 적용 관련하여 질의가 들어와 다른 감독관님들의 의견을 구합...
3       [\n, [시정지시서가 사업장의 폐문부재 등으로 반송되었을 경우, 범죄인지 및 과태...
4       [\n, [질문드립니다.], \n, [[]], \n, [[- 진정요지: 임금체불]]...
                              ...                        
4741    <p>또 생각나는게 있어서 올립니다.</p>\r\n<p>어떻게 생각하면 간단한 것 ...
4742    <p>좋은 공간이 생겼네요. </p>\r\n<p>평소에 궁금하던 것들에 대해&nbs...
4743    <p>회사에서&nbsp;근무하는 근로자가 근로자인지 아닌지 판단할 수 있는 기준을 ...
4744    <p>비정규직에 대한 차별처우 금지영역, 합리적 차별 내용, 차별시정절차 등 차별시...
4745                <p>일용직 주휴수당지급관련 답변을 구합니다.</p>\r\n    
Name: 질문내용_body, Length: 4746, dtype: object

In [11]:
df1['질문내용_html']

0       <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...
1       <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...
2       <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...
3       <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...
4       <html xml:lang="ko" lang="ko">\r\n<head>\r\n<s...
                              ...                        
4741    <p>또 생각나는게 있어서 올립니다.</p>\r\n<p>어떻게 생각하면 간단한 것 ...
4742    <p>좋은 공간이 생겼네요. </p>\r\n<p>평소에 궁금하던 것들에 대해&nbs...
4743    <p>회사에서&nbsp;근무하는 근로자가 근로자인지 아닌지 판단할 수 있는 기준을 ...
4744    <p>비정규직에 대한 차별처우 금지영역, 합리적 차별 내용, 차별시정절차 등 차별시...
4745                <p>일용직 주휴수당지급관련 답변을 구합니다.</p>\r\n    
Name: 질문내용_html, Length: 4746, dtype: object

In [12]:
# HTML 테이블을 추출하는 함수
def extract_table_from_html(html_content):
    try:
        # HTML 데이터에서 모든 테이블을 읽어옴
        tables = pd.read_html(StringIO(html_content))
        if tables:
            # 첫 번째 테이블만 추출
            return tables[0]
        else:
            return None
    except Exception as e:
        # 테이블이 없거나 오류가 발생하면 None 반환
        return None

# 질문내용_html 컬럼에서 테이블 추출하여 새로운 컬럼에 저장
df1['질문내용_table'] = df1['질문내용_html'].apply(extract_table_from_html)

In [ ]:
df1.columns

In [ ]:
df1[df1['등록자']=='김정래']

In [ ]:
df1[df1['질문내용_table'].notna()]

In [13]:
selected_columns = ['분야', '질문명', '등록일', '등록자' ,'답변수', '조회수', '질문내용_html', '질문내용_body']

In [14]:
df1[selected_columns].to_csv("output.csv", index=False, encoding="utf-8-sig", escapechar="\\")